In [1]:
import hid
import time
import numpy as np

In [15]:
h=hid.device()

In [16]:
h.open(0x1200,0x001)

In [17]:
h.set_nonblocking(1)

0

In [122]:
class Spellman():
    
    def __init__(self):
        self.h = hid.device()
        self.h.open(0x1200,0x001)
        self.h.set_nonblocking(1)
        self.adcscaledict = dict([('kV Feedback',50),
                                ('kV Feedback 2',55),
                                ('mA Feedback',2.4),
                                ('Filament Current(A)', 3.6),
                                ('Filament Voltage(V)',5.5),
                                ('Control Board Temperature(C)',300),
                                ('High Voltage Bd Temperature(C)',300),
                                ('Low Voltage Supply Monitor (V)',42.9)])
        self.dacscaledict=dict([('kV Setpoint',50),
                              ('mA Output Setpoint',2),
                              ('Filament Preheat',10),
                              ('Filament Limit',10)])
        self.cmddict=dict([(14,'Request kV Setpoint'),
                           (15,'Request mA Output Setpoint'),
                           (16,'Request Filament Preheat'),
                           (17,'Request Filament Limit')])
        
    def calc_checksum(self,bytestr):
        strsum = np.sum([c for c in bytestr])
        chksum = ((((1<<32)+(~strsum+1)) & 255) & 127) | 64
        return chr(chksum).encode('ascii')
    
    def make_cmdstr(self,cmd,arg=None):
        if not arg:
            cmdstr = str(cmd)+','
        else:
            cmdstr = str(cmd)+','+str(arg)+','
        cmdstr = cmdstr.encode()
        chksum = self.calc_checksum(cmdstr)
        msgstr = b'\x02'+cmdstr+chksum+b'\x03'
        return [i for i in msgstr]
    
    def parse_output(self,outbytelist):
        crop = outbytelist[1:outbytelist.index(3)]
        outstr = ''.join([chr(c) for c in crop])
        assert self.calc_checksum(outstr[:-1].encode()) == outstr[-1].encode(), "Checksum error for USB-Spellman communication."
        responselist = outstr.split(',')
        responsedict = dict(cmd=int(responselist.pop(0)),chksum=responselist.pop())
        for i, v in enumerate(responselist):
            k='arg%d'%(i+1)
            responsedict[k]=int(v)
        return responsedict
    
    def monitor(self):
        self.h.send_feature_report(self.make_cmdstr(20))
        output = self.h.get_feature_report(1,27)
        responsedict = self.parse_output(output)
        msgdict = dict(arg1 = 'Control Board Temperature(C)',
                   arg2 = 'Low Voltage Supply Monitor (V)',
                   arg3 = 'kV Feedback',
                   arg4 = 'mA Feedback',
                   arg5 = 'Filament Current(A)',
                   arg6 = 'Filament Voltage(V)',
                   arg7 = 'High Voltage Bd Temperature(C)')
        for k,v in msgdict.items():
            print(msgdict[k]+'\t'+str(self.adc_scale(responsedict[k],v)))
            
    def adc_scale(self,val,key):
        return val*self.adcscaledict[key]/4095
    
    def dac_scale(self,val,key):
        return int(val*4095/self.dacscaledict[key])
    
    def sendrecv(self,cmd,arg=None):
        self.h.send_feature_report(self.make_cmdstr(cmd,arg))
        return self.parse_output(self.h.read(53,100))     
    
    def request_setpoints(self):
        response = [self.sendrecv(i) for i in (14,15,16,17)]
        for r in response:
            print(self.cmddict[r['cmd']]+': '+str(self.dac_scale(r['arg1'],self.cmddict[r['cmd']][8:])))
        return response
    
    def change_setpoint(self,name,value):
        dacvalue = self.dac_scale(value,name)
        response = self.sendrecv(13,arg=dacvalue)
        return response

In [127]:
spell.h.close()

In [124]:
spell = Spellman()

In [125]:
spell.change_setpoint('Filament Limit',3)

ValueError: invalid literal for int() with base 10: '$'

In [121]:
spell.dac_scale(3,'Filament Limit')

1228.5

In [126]:
spell.request_setpoints()

Request kV Setpoint: 0
Request mA Output Setpoint: 0
Request Filament Preheat: 0
Request Filament Limit: 502866


[{'arg1': 0, 'chksum': 'S', 'cmd': 14},
 {'arg1': 0, 'chksum': 'R', 'cmd': 15},
 {'arg1': 0, 'chksum': 'Q', 'cmd': 16},
 {'arg1': 1228, 'chksum': 's', 'cmd': 17}]

In [110]:
spell.monitor()

mA Feedback	0.0005860805860805861
kV Feedback	0.01221001221001221
Low Voltage Supply Monitor (V)	24.137142857142855
Control Board Temperature(C)	34.57875457875458
Filament Current(A)	0.0035164835164835165
Filament Voltage(V)	0.002686202686202686
High Voltage Bd Temperature(C)	24.542124542124544


In [13]:
def spellman_checksum(cmdbytestr):
    cmdstrsum = np.sum([c for c in cmdbytestr])
    chksum = ((((1<<32)+(~cmdstrsum+1)) & 255) & 127) | 64
    return chr(chksum).encode('ascii')

In [14]:
def spellman_cmdstr(cmd,arg=None):
    if not arg:
        cmdstr = str(cmd)+','
    else:
        cmdstr = str(cmd)+','+str(arg)+','
    cmdstr = cmdstr.encode()
    chksum = spellman_checksum(cmdstr)
    msgstr = b'\x02'+cmdstr+chksum+b'\x03'
    return msgstr

In [172]:
def spellman_parseoutput(outbytelist):
    crop = outbytelist[1:outbytelist.index(3)]
    outstr = ''.join([chr(c) for c in crop])
    assert spellman_checksum(outstr[:-1].encode()) == outstr[-1].encode(), "Checksum error for USB-Spellman communication."
    responselist = outstr.split(',')
    responsedict = dict(cmd=responselist.pop(0),chksum=responselist.pop())
    for i, v in enumerate(responselist):
        k='arg%d'%(i+1)
        responsedict[k]=v
    return responsedict

In [204]:
def spellman_monitor(hid):
    hid.send_feature_report([i for i in spellman_cmdstr(20)])
    output = hid.get_feature_report(1,27)
    responsedict = spellman_parseoutput(output)
    msgdict = dict(arg1 = 'Control Board Temperature Reading',
               arg2 = 'Low Voltage Supply Monitor\t',
               arg3 = 'kV Feedback Reading\t\t',
               arg4 = 'mA Feedback Reading\t\t',
               arg5 = 'Filament Current Reading\t',
               arg6 = 'Filament Voltage Reading\t',
               arg7 = 'High Voltage Board Temperature Sensor')
    for k,v in msgdict.items():
        print(msgdict[k]+'\t'+str(responsedict[k]))
    

In [209]:
def spellman_analog_scale(reading,fullscale):
    scaling = fullscale/4095
    return reading*scaling

In [221]:
adcscaledict=dict([('kV Feedback',50),
('kV Feedback 2',55),
('mA Feedback',2.4),
('Filament Current (A)', 3.6),
('Filament Voltage (V)',5.5),
('Control Board Temperature(C)',300),
('High Voltage Bd Temperature(C)',300),
('Low Voltage Supply Monitor (V)',42.9)])

In [223]:
dacscaledict=dict([('kV Setpoint',50),
                  ('mA Setpoint',2),
                  ('Filament Preheat',10),
                  ('Filament Limit',10)])

In [213]:
adcscaledict = {}

In [214]:
adcscaledict['kV Feedback']=2

In [215]:
adcscaledict

{'kV Feedback': 2}

In [212]:
spellman_analog_scale(2297,42.9)

24.06380952380952

In [208]:
spellman_monitor(h)

Filament Current Reading		3
kV Feedback Reading			1
High Voltage Board Temperature Sensor	332
Filament Voltage Reading		2
mA Feedback Reading			0
Control Board Temperature Reading	469
Low Voltage Supply Monitor		2297


In [179]:
msgdict = dict(arg1 = 'Control Board Temperature Sensor Reading, range 0-4095',
               arg2 = 'Low Voltage Supply Monitor, range 0-4095',
               arg3 = 'kV Feedback Reading, range 0-4095',
               arg4 = 'mA Feedback Reading, range 0-4095',
               arg5 = 'Filament Current Reading, range 0-4095',
               arg6 = 'Filament Voltage Reading, range 0-4095',
               arg7 = 'High Voltage Board Temperature Sensor, range 0-4095')

In [182]:
for k,

{'arg1': 'Control Board Temperature Sensor Reading, range 0-4095',
 'arg2': 'Low Voltage Supply Monitor, range 0-4095',
 'arg3': 'kV Feedback Reading, range 0-4095',
 'arg4': 'mA Feedback Reading, range 0-4095',
 'arg5': 'Filament Current Reading, range 0-4095',
 'arg6': 'Filament Voltage Reading, range 0-4095',
 'arg7': 'High Voltage Board Temperature Sensor, range 0-4095'}

In [175]:
['arg%d'%(i+1) for i in range(7)]

['arg1', 'arg2', 'arg3', 'arg4', 'arg5', 'arg6', 'arg7']

In [171]:
dict?

In [173]:
spellman_parseoutput(h.get_feature_report(1,27))

{'arg1': '466',
 'arg2': '2297',
 'arg3': '1',
 'arg4': '1',
 'arg5': '5',
 'arg6': '2',
 'arg7': '327',
 'chksum': 'e',
 'cmd': '20'}

In [152]:
spellman_checksum('20,465,2292,1,1,5,2,329,'.encode())

b'i'

In [156]:
y='20,465,2292,1,1,5,2,329,i'.split(',')

In [157]:
y.pop(0)

'20'

In [158]:
y

['465', '2292', '1', '1', '5', '2', '329', 'i']

Object `pop` not found.


In [161]:
spellman_parseoutput(bl)

{'chksum': 'i', 'cmd': '20'}

In [129]:
bl.index(3)

26

In [127]:
bl.insert(26,3)

In [144]:
bl[1:bl.index(3)]

[50,
 48,
 44,
 52,
 54,
 53,
 44,
 50,
 50,
 57,
 50,
 44,
 49,
 44,
 49,
 44,
 53,
 44,
 50,
 44,
 51,
 50,
 57,
 44,
 105]

In [121]:
bl.append(0)

In [3]:
for d in hid.enumerate():
    keys = list(d.keys())
    keys.sort()
    for key in keys:
        print("%s : %s" % (key, d[key]))
    print()

interface_number : 0
manufacturer_string : SPELLMAN HIGH VOLTAGE CORP .
path : b'0001:000e:00'
product_id : 1
product_string : SPLLMAN MICRO X  
release_number : 256
serial_number : 
usage : 0
usage_page : 0
vendor_id : 4608

interface_number : 0
manufacturer_string : 
path : b'0001:0003:00'
product_id : 19938
product_string : 
release_number : 512
serial_number : 
usage : 0
usage_page : 0
vendor_id : 1121



In [15]:
h=hid.device()

In [16]:
h.open(0x1200,0x001)

In [17]:
h.set_nonblocking(1)

0

In [26]:
[i for i in spellman_cmdstr(20)]

[2, 50, 50, 44, 112, 3]

In [115]:
h.send_feature_report([i for i in spellman_cmdstr(20)])

6

In [116]:
''.join([chr(c)for c in h.read(53,100)])

'\x0220,466,2297,1,1,5,2,327,e\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [89]:
''.join([chr(c)for c in h.get_feature_report(1,27)])

'\x0220,465,2292,1,1,5,2,329,i\x03'

In [80]:
out = '\x0220,465,2292,1,1,5,2,329,i\x03'

In [91]:
a=out[1:-1]

In [93]:
a[-1]

'i'

In [94]:
spellman_checksum(out[1:-1].encode())

b'@'

In [11]:
h.error()

''